# Exercise 05. (Take-home) Advanced Topic: Adding an in-game Recommender using RLlib

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives
In this this tutorial, you will learn:

 * 

### Exercises

1. 

### Homework

1. 

### References

* 